Load images

In [ ]:
import glob

In [ ]:
good_images = glob.glob("/root/data/lice-data/crops/blom-kjeppevikholmen/*/2/*.jpg")
print("Number of good images {}".format(len(good_images)))
bad_images_raw = glob.glob("/root/data/priority_queue/images/*/*.jpg")
bad_images = []
names = []
for im in bad_images_raw:
    name = im.split("/")[-1]
    if name not in names:
        names.append(name)
        bad_images.append(im)
print("Number of bad images {}".format(len(bad_images)))

Load model

In [ ]:
import keras

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
model = keras.models.load_model("/root/data/priority_queue/models/draft_0.h5")

Generator

In [ ]:
import random
import numpy as np
import cv2

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, good_images, bad_images, batch_size=16, dim=(224, 224, 3)):
        'Initialization'
        self.good_images = good_images
        self.batch_size = batch_size
        self.bad_images = bad_images
        self.dim = dim
        self.on_epoch_end()
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int((len(self.good_images) + len(self.bad_images)) / self.batch_size)
    
    def __getitem__(self, index):
        'Generate one batch of data'
        xbatch = []
        ybatch = []
        for i in range(self.batch_size):
            coin = np.random.rand()
            if coin > 0.5:
                path = np.random.choice(self.good_images)
            else:
                path = np.random.choice(self.bad_images)
        
            image = cv2.imread(path)
            image = cv2.resize(image, (self.dim[0], self.dim[1]))
            xbatch.append(image)
            ybatch.append(np.round(coin))

        return np.array(xbatch), np.array(ybatch)
    
    def on_epoch_end(self):
        random.shuffle(self.good_images)
        random.shuffle(self.bad_images)

In [ ]:
gen = DataGenerator(good_images, bad_images)

Prediction

In [ ]:
xb, yb = gen[0]

In [ ]:
ypred = model.predict_on_batch(xb)

In [ ]:
from time import time

In [ ]:
start = time()
# model.predict_on_batch(np.expand_dims(xb[0, ...], axis=0))
image = cv2.imread(good_images[0])
image = cv2.resize(image, (224, 224))
model.predict_on_batch(np.expand_dims(image, axis=0))
# ypred = model.predict_on_batch(xb)
end = time()
print(end-start)

Compare

In [ ]:
batch_acc = np.sum(yb == np.round(ypred).squeeze()) / 16.0
print("Batch accuracy {}".format(batch_acc))

Let's experiment

In [ ]:
# take a good image
path = np.random.choice(good_images)
image = cv2.imread(path)
image = cv2.resize(image, (224, 224))
pred = model.predict_on_batch(np.expand_dims(image, axis=0))[0]
print("prediction {}".format(pred))

In [ ]:
from imgaug import augmenters as iaa
import matplotlib.pyplot as plt

In [ ]:
aug = iaa.GaussianBlur((0, 3.0), name="GaussianBlur")

In [ ]:
augmented = aug.augment_image(image)

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(image)
ax[1].imshow(augmented)
plt.show()

Blur experiment

In [ ]:
predictions = []
_range = np.arange(0.0, 3.0, 0.05)
for i in _range:
    aug = iaa.GaussianBlur((i, i), name="GaussianBlur")
    augmented = aug.augment_image(image)
#     plt.imshow(augmented)
#     plt.show()
    pred = model.predict_on_batch(np.expand_dims(augmented, axis=0))[0]
#     print("prediction {}".format(pred))
    predictions.append(pred)

In [ ]:
plt.plot(_range, predictions)
plt.xlabel("Blur amount")
plt.ylabel("Probability of being a good image")
plt.show()

Darkness experiment

In [ ]:
predictions = []
_range = np.arange(0, -100, -2)
for i in _range:
    aug = iaa.Add((i, i))
    augmented = aug.augment_image(image)
#     plt.imshow(augmented)
#     plt.show()
    pred = model.predict_on_batch(np.expand_dims(augmented, axis=0))[0]
#     print("prediction {}".format(pred))
    predictions.append(pred)

In [ ]:
plt.plot(_range, predictions)
plt.xlabel("Darkness amount")
plt.ylabel("Probability of being a good image")
plt.xlim(_range[0], _range[-1]) 
plt.show()

Obstruction experiment

Bad crop experiment